# HW3 Solutions


## Import libraries


##### `nltk`

- **Description**: `Natural Language Toolkit`, I use it for tokenization, stemming , remove stop words and lemmatizer.

##### `glob`

- **Description**: Just for reading documents name in ./docs directory

##### `string`

- **Description**: I used from punctuations attribute on this package

##### `json`

- **Description**: For encoding and decoding inverted indexes when we want to write in disk or read from disk

##### `math`

- **Description**: For calculation, log and ...

##### `shutil | os`

- **Description**: I imported these for creating and removing disk directory


In [1]:
import nltk
import glob
import string
import json
import math
import shutil
import os
# nltk.download('stopwords')
# nltk.download('wordnet')

The `gamma code` is a universal code that is used to encode a sequence of positive integers. It is developed by Peter Elias. It is most useful when the upper-bound of integers cannot be determined beforehand and we use it for optimization on docement ids.

##### Methods

##### `gammaDecoder(input : str) -> int`

- **Description**: i this section I get encoded gamma code and decode that and finally i will return the result as a integer.

##### `gammaEncoder(input : int) -> str`

- **Description**: I will encode the integer that you gave me as input


## Gamma Coding And Opthional section


In [2]:
class GammaCoding:
        def gammaDecoder(input : str) -> int:
            if(input) == '0' : return 0

            input = list(input) 
            k = 0
            while input[k]=='0': 
                k += 1

            input = input[k:2*k+1] 
            input.reverse() 

            res = 0
            for ind, s in enumerate(input): 
                if s == '1': 
                    res += math.pow(2, ind) 
            return int(res)


        def gammaEncoder(input : int) -> str:
            log2 = lambda x: math.log(x, 2) 

            if not input:
                return '0'
            
            n = int(log2(input))
            b = input - 2**n

            return n *'0'+'1' + ('{0:0%db}' % n).format(b)  

## Utils


The `Utils` class contains utility methods for set operations like intersection and union and proximity distance calculations that is a little custom function(maybe I should change it's name) and in this practice I implemeted merge and I use it.

#### Methods

##### `merge(l1: list, l2: list) -> list`

- **Description**: This function will merge two sorted posting list, as the way that we construct our postings we know that they are sorted and $L2 < L1$.


In [3]:
class Utils:
    def __init__(self) -> None:
        pass

    def merge(l1: list, l2: list)->list:
        res = []
        i, j =0, 0
        while(i<len(l1) and j<len(l2)):
            if GammaCoding.gammaDecoder(l1[i]) < GammaCoding.gammaDecoder(l2[j]) : 
                res.append(l1[i])
                i+=1
            else:
                res.append(l2[j])
                j+=1

        if(i<len(l1)) : 
            res+=l1[i:]
        else : 
            res+= l2[j:]

        # save differences of doc ids for gamma coding instead of the id 
        if len(res) >1:
            res[1] = GammaCoding.gammaEncoder(abs(GammaCoding.gammaDecoder(res[0])-GammaCoding.gammaDecoder(res[1])))
        for i in range(2, len(res)):
            res[i] = GammaCoding.gammaEncoder(abs((GammaCoding.gammaDecoder(res[i-1]) + GammaCoding.gammaDecoder(res[0])) - GammaCoding.gammaDecoder(res[i])))

        return res     

## Load Documents


The `LoadDocuments` class is responsible for loading and managing a collection of documents. If you want to change the document path please cgange path variable.

##### Properties

- **_`DOC_ID_MAPPER`_** _(class variable)_: A dictionary mapping document names to their respective IDs, Be care full, becuse it depends on your documents order and docIds started from zero.

##### Methods

##### `__init__(self)`

- **Description**: Initializes a `LoadDocuments` object with a default path to load text that you can change it base on your docs directory path and an empty `memoryDocumentCollection`. that is a dictionary and I will set initial data of problem.

##### `removeLoadedDocumentsFromMemory(self)`

- **Description**: I will destruct `memoryDocumentCollection` property.

##### `loadDocumentFromDisk(self)`

- **Description**: Loads text documents from the disk and populates the `memoryDocumentCollection` property.

##### `buildDocumentIDMapper(self)`

- **Description**: Builds a document ID mapper by associating each document's name with a unique ID and populates the `DOC_ID_MAPPER` dictionary, I will print this mapper data for checking my results base on this mapper.


In [4]:
class LoadDocuments:      

    DOC_ID_MAPPER = {}

    def __init__(self) -> None:
        LoadDocuments.removeDisk()
        LoadDocuments.createDisk()
        self.path = "./docs/*.txt"
        self.limitationNumberOfDocumentsInMemory = 3
        self.allDocumentNames = glob.glob(self.path)
        self.memoryDocumentCollection = {}
        self.lastLoadedDocumentId = 0

    def createDisk() -> None:
        path = os.path.join('./', 'Disk') 
        try:
            os.mkdir(path)
        except OSError:
            pass

    def removeDisk() -> None:
        try:
            shutil.rmtree('./Disk')
        except OSError:
            pass   

    def removeLoadedDocumentsFromMemory(self)->None:
        self.memoryDocumentCollection = {}

    def loadDocumentFromDisk(self)->None:
        print(self.allDocumentNames[self.lastLoadedDocumentId : self.lastLoadedDocumentId + self.limitationNumberOfDocumentsInMemory])
        for documentName in self.allDocumentNames[self.lastLoadedDocumentId : self.lastLoadedDocumentId + self.limitationNumberOfDocumentsInMemory] :
            document = open(documentName, "r", encoding = 'cp1252')
            self.memoryDocumentCollection[documentName] = document.read()
        self.lastLoadedDocumentId += self.limitationNumberOfDocumentsInMemory    

    def buildDocumentIDMapper(self)->None:
        for ind, documentName in enumerate(self.memoryDocumentCollection):
            self.DOC_ID_MAPPER[documentName] = ind + (self.lastLoadedDocumentId-self.limitationNumberOfDocumentsInMemory) + 1
            print("docId : " , ind + (self.lastLoadedDocumentId-self.limitationNumberOfDocumentsInMemory) + 1 , ' ->  docName: ', documentName[7:])

## Document Preprocessing


The `DocumentPreProcessor` class extends the functionality of the `LoadDocuments` class by providing methods to preprocess text documents. These methods perform various text processing tasks, such as converting text to lowercase, tokenization, removing punctuations, removing stop words, stemming, and lemmatization.

#### Methods

##### `__init__(self)`

- **Description**: Initializes a `DocumentPreProcessor` object by calling the constructor of the base class `LoadDocuments`.

##### `convertToLower(self)`

- **Description**: Converts all text in the loaded documents to lowercase.

##### `tokenizer(self)`

- **Description**: Tokenizes the text in the loaded documents using a regular expression pattern.

##### `removePunctuations(self)`

- **Description**: Removes punctuations from the tokenized text in the loaded documents.

##### `removeStopWords(self)`

- **Description**: Removes stop words from the tokenized text in the loaded documents using NLTK's English stop words list.

##### `stemming(self)`

- **Description**: Applies stemming to the tokenized text in the loaded documents using the Porter Stemmer algorithm from NLTK.

##### `lemmatizer(self)`

- **Description**: Applies lemmatization to the tokenized text in the loaded documents using NLTK's WordNet Lemmatizer.


In [5]:
class DocumentPreProcessor(LoadDocuments):
    def __init__(self)->None:
        super().__init__()

    def convertToLower(self)->None:
        for documentName, document in self.memoryDocumentCollection.items():
            self.memoryDocumentCollection[documentName] = document.lower()

    def tokenizer(self)->None:
        pattern = r'\d{1,3}(?:,\d{3})*(?:\.\d+)?|\w+'
        for documentName, document in self.memoryDocumentCollection.items():
            self.memoryDocumentCollection[documentName] = nltk.tokenize.regexp_tokenize(document, pattern)

    def removePunctuations(self)->None:
        for documentName, document in self.memoryDocumentCollection.items():
            for ind, term in enumerate(document):
                self.memoryDocumentCollection[documentName][ind] = "".join([i for i in term if i not in string.punctuation])

    def removeStopWords(self)->None:
        stopwords = nltk.corpus.stopwords.words('english')
        for documentName, document in self.memoryDocumentCollection.items():
            self.memoryDocumentCollection[documentName] = [i for i in document if i not in stopwords]

    def stemming(self)->None:
        porter_stemmer = nltk.stem.porter.PorterStemmer()
        for documentName, document in self.memoryDocumentCollection.items():
            self.memoryDocumentCollection[documentName] = [porter_stemmer.stem(term) for term in document]

    def lemmatizer(self)->None:
        wordnet_lemmatizer = nltk.stem.WordNetLemmatizer()
        for documentName, document in self.memoryDocumentCollection.items():
            self.memoryDocumentCollection[documentName] = [wordnet_lemmatizer.lemmatize(term) for term in document]

## Inverted Index


The `InvertedIndex` class will generate a invertedIndexex that maps terms to the documents that contain them, I will save all positions for proximate search .

#### Properties

- **`invertedIndex`**: A dictionary representing the inverted index where terms are mapped to document IDs and their positions.

#### Methods

##### `__init__(self)`

- **Description**: Initializes an `InvertedIndex` object and call base class constructor.

##### `buildInvertedIndex(self)`

- **Description**: Builds the inverted index by iterating through preprocessed documents and mapping terms to document IDs and their positions.

##### `removeInvertedIndexFromMemory(self)`

- **Description**: deconstruct `InvertedIndex` and remove its data from memory.

##### `build(self) -> None`

- **Description**: Initiates the preprocessing steps by loading documents, building document ID mapper, converting to lowercase, tokenizing, removing punctuations, removing stop words, stemming, lemmatizing, and finally, building the inverted index.


In [6]:
class InvertedIndex(DocumentPreProcessor):
    def __init__(self) -> None:
        super().__init__()
        self.invertedIndex = {}    

    def buildInvertedIndex(self)->None:
        for documentName, document in self.memoryDocumentCollection.items():
            for ind, term in enumerate(document):
                if term not in self.invertedIndex:
                    self.invertedIndex[term] = []
  
                self.invertedIndex[term].append(GammaCoding.gammaEncoder(self.DOC_ID_MAPPER[documentName]))
                self.invertedIndex[term] = list(set(self.invertedIndex[term]))

    def removeInvertedIndexFromMemory(self)->None:            
        self.invertedIndex = {}

    def execute(self)->None:
        self.removeLoadedDocumentsFromMemory()
        self.removeInvertedIndexFromMemory()
        self.loadDocumentFromDisk()
        self.buildDocumentIDMapper()
        self.convertToLower()
        self.tokenizer()
        self.removePunctuations()
        self.removeStopWords()
        self.stemming()
        self.lemmatizer()
        self.buildInvertedIndex()      

## BSBI


The `InvertedIndex` class will generate a invertedIndexex that maps terms to the documents that contain them, I will save all positions for proximate search .

#### Properties

- **`finalInvertedIndex`**: A dictionary representing the inverted index where terms are mapped to a list of document IDs, this is the final inverted index that is on memory.

- **`numberOfBlocks`**: A integer representing number of all blocks that exist in disk

#### Methods

##### `__init__(self)`

- **Description**: Initializes an `finalInvertedIndex` object and `numberOfBlocks` and call base class constructor.

##### `writeToDisk(self, inputData: dict) -> None`

- **Description**: Write current inverted index to disk.

##### `readBlock(self, blockNo: int)-> dict`

- **Description**: Read the i'th block from disk

##### `build(self) -> None`

- **Description**: it's a executer for BSBI that will load all blocks one by one and merge them to construct final inverted index.

##### `mergeBlocks(self) | merge(self) -> None`

- **Description**: It read the first block and insert it to `finalInvertedIndex` then I will read blocks one by one and merge them to `finalInvertedIndex`


In [7]:
class BSBI(InvertedIndex):

    def __init__(self) -> None:
        super().__init__()
        self.finalInvertedIndex = {}
        self.numberOfBlocks = 0

    def build(self) -> None:
        while(self.lastLoadedDocumentId < len(self.allDocumentNames)):
            self.numberOfBlocks+=1
            self.execute()
            self.writeToDisk(self.invertedIndex)

        self.mergeBlocks()     

    def writeToDisk(self, inputData: dict) -> None:
        with open("./Disk/document%i.txt"%(self.numberOfBlocks), 'w') as convert_file: 
            convert_file.write(json.dumps(inputData))

    def readBlock(self, blockNo: int)-> dict:
        with open("./Disk/document%i.txt"%(blockNo), "r") as f: 
            data = f.read() 
        return json.loads(data) 
    
    def merge(self, invertedIndex : dict) -> None:
        for term, documents in invertedIndex.items():
            if term not in self.finalInvertedIndex:
                self.finalInvertedIndex[term] = documents
            else:
                self.finalInvertedIndex[term] = Utils.merge(self.finalInvertedIndex[term], documents)
                
    def mergeBlocks(self) -> None:
        print("block numbers" , self.numberOfBlocks)

        self.finalInvertedIndex = self.readBlock(1)

        for i in range(2, self.numberOfBlocks):
            self.merge(self.readBlock(i+1))

    

In [8]:
BSBI = BSBI()
BSBI.build()

['./docs/Jerry Decided To Buy a Gun.txt', './docs/Rentals at the Oceanside Community.txt', './docs/Gasoline Prices Hit Record High.txt']
docId :  1  ->  docName:  Jerry Decided To Buy a Gun.txt
docId :  2  ->  docName:  Rentals at the Oceanside Community.txt
docId :  3  ->  docName:  Gasoline Prices Hit Record High.txt
['./docs/Cloning Pets.txt', './docs/Crazy Housing Prices.txt', './docs/Man Injured at Fast Food Place.txt']
docId :  4  ->  docName:  Cloning Pets.txt
docId :  5  ->  docName:  Crazy Housing Prices.txt
docId :  6  ->  docName:  Man Injured at Fast Food Place.txt
['./docs/A Festival of Books.txt', './docs/Food Fight Erupted in Prison.txt', './docs/Better To Be Unlucky.txt']
docId :  7  ->  docName:  A Festival of Books.txt
docId :  8  ->  docName:  Food Fight Erupted in Prison.txt
docId :  9  ->  docName:  Better To Be Unlucky.txt
['./docs/Sara Went Shopping.txt', './docs/Freeway Chase Ends at Newsstand.txt', './docs/Trees Are a Threat.txt']
docId :  10  ->  docName:  Sar

### Final Inverted Indes In My memory:


In [9]:
print(BSBI.finalInvertedIndex)

{'jerri': ['10'], 'baldwin': ['10'], '30': ['011', '010', '00101'], 'year': ['011', '10', '010', '10', '010', '0', '0', '011', '00111', '00101', '00110'], 'old': ['10', '00110', '00101', '010', '0', '0001100', '010'], 'manag': ['011', '10', '011', '00101'], 'pizza': ['10'], 'restaur': ['10'], 'live': ['10', '00101', '011', '00100', '0001000'], 'apart': ['10', '0001101'], 'one': ['010', '10', '010', '00100', '011', '00100', '00111', '00110', '00110'], 'mile': ['011', '10', '10', '0001011'], 'north': ['10', '0001010'], 'walk': ['10'], 'work': ['011', '010', '010', '10', '0001011'], 'rain': ['10', '0001110'], 'took': ['10'], 'bu': ['10', '0001010'], 'love': ['10', '00110'], 'gangster': ['10'], 'movi': ['10', '00110'], 'new': ['10', '0001101'], 'came': ['10', '00101', '00100', '0001000', '00110', '00111'], 'would': ['010', '10', '10', '0', '10', '0001010', '011', '0001001'], 'go': ['011', '010', '00101', '10', '0001001', '010'], 'theater': ['10'], 'watch': ['10', '00101', '0001001'], 'thre

## Report:


### Steps:

$1.$ For this project at first I revied my previous home works, and most of my codes was compatible and I use from them.

&nbsp;

$2.$ Then I read about gamma coding in geeksforgeeks and I implemeted it.(it was not challenging, we had it's defenition and it was simple)

&nbsp;

$3.$ Then I read our slides about BSBI that said for memory limitation we can't load all documents to ram, so we use disk. We have some blocks in our disk that every block has a little inverted index, in my code I will load three document from disk to memory then I will generate it's inverted index and then write it to a block, then in BSBI we will read these blocks that every one of them is a inverted index, then I will merge them.

&nbsp;

$4.$ There are some Notes about BSBI :

1. We assumed that we can load and save final inverted index to our memory.
2. As I read all documents one by one, so my posting lists in all bocks are sorted, so I don't need sort them.
3. As you know every inverted index that I saved to disk are small so I didn't save differences of doc Ids to my posting lists, but in final inverted index I save differences or gaps between document IDs.

&nbsp;

$5.$ I assembled all these component and in this step I forced to write a Util for giving some functionality to me.

&nbsp;

$6.$ I faced a issue in my code, in HW1 in inverted index we saved positions but here we did'nt need to them, so I deleted all of them and I will keep docId in posting lists

&nbsp;

$7.$ In last step I write a documentation for my code, but for better style and a formal language I gave them and my classes to chatGpt I helped me to generate documents, at last I revise all things and merge all of them and write my document.


## References:


1. Our course slides
2. https://www.nltk.org/
3. https://chat.openai.com/
4. https://www.geeksforgeeks.org/
